# Liveness Detection (Anti-Spoofing) — вариант 57

## Анализ предметной области и постановка задачи

Современные системы биометрической аутентификации уязвимы к атакам подмены личности
(presentation attacks), при которых злоумышленник использует фотографию, видеозапись
или экран для обмана системы распознавания лиц. В связи с этим актуальной является задача
**детекции живости лица (liveness detection)** — определения, принадлежит ли изображение
реальному человеку или является спуфинг-атакой.

**Цель работы:** разработка и экспериментальная проверка программного прототипа
детекции живости лица на основе анализа текстурных признаков изображения.

**Задачи:**
- проанализировать методы детекции живости;
- реализовать алгоритм извлечения признаков;
- обучить классификатор;
- оценить эффективность метода;
- сформулировать выводы о применимости технологии.


## Обзор существующих методов детекции живости

Существующие подходы к liveness detection можно условно разделить на несколько групп:

1. **Анализ движений и поведения** (моргание, микродвижения, реакция на команды).
2. **Текстурный анализ** — выявление отличий текстуры кожи от печатных или экранных изображений.
3. **Глубинные методы** — использование нейронных сетей (CNN, Transformer).
4. **Мультимодальные подходы** — объединение RGB, depth, IR.

В рамках учебной работы выбран **текстурный метод на основе LBP**,
как интерпретируемый и вычислительно простой.


## Описание реализации выбранного алгоритма

### Используемые данные

Датасет содержит изображения лиц, разделённые на обучающую (`train_img`) и тестовую (`test_img`) выборки.
Класс изображения определяется по имени файла:
- `_real` — живое лицо (label = 1);
- `_fake` — спуфинг-атака (label = 0).

### Признаки
Для извлечения признаков используется **Local Binary Patterns (LBP)** —
метод анализа локальной текстуры изображения.

### Классификатор
В качестве классификатора используется **логистическая регрессия**.


In [ ]:
# Импорты
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay, ConfusionMatrixDisplay


In [ ]:
IMG_SIZE = (128, 128)

def load_images(path, max_per_class=150):
    X, y = [], []
    files = [f for f in os.listdir(path) if f.endswith('.jpg')]
    real = [f for f in files if '_real' in f][:max_per_class]
    fake = [f for f in files if '_fake' in f][:max_per_class]

    for fname in real + fake:
        img = cv2.imread(os.path.join(path, fname), cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE)
        label = 1 if '_real' in fname else 0
        X.append(img)
        y.append(label)
    return np.array(X), np.array(y)


In [ ]:
def lbp_histogram(gray):
    h, w = gray.shape
    lbp = np.zeros_like(gray)
    offsets = [(-1,-1), (-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1)]
    for y in range(1, h-1):
        for x in range(1, w-1):
            center = gray[y, x]
            code = 0
            for i, (dy, dx) in enumerate(offsets):
                code |= (gray[y+dy, x+dx] >= center) << i
            lbp[y, x] = code
    hist, _ = np.histogram(lbp.ravel(), bins=256, range=(0,256), density=True)
    return hist


## Экспериментальные исследования

Обучение выполняется на `train_img`, оценка — на независимой тестовой выборке `test_img`.


In [ ]:
X_train_img, y_train = load_images('dataset/train_img/color')
X_test_img, y_test = load_images('dataset/test_img/color')

X_train = np.array([lbp_histogram(img) for img in X_train_img])
X_test = np.array([lbp_histogram(img) for img in X_test_img])


In [ ]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print('ROC-AUC:', roc_auc_score(y_test, y_proba))

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.show()

ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred,
    display_labels=["spoof", "live"],
    cmap="Blues"
)

## Выводы и рекомендации по применению

В ходе работы был разработан программный прототип детекции живости лица
на основе текстурных признаков LBP и логистической регрессии.

**Выводы:**
- метод демонстрирует способность различать живые лица и спуфинг-атаки;
- использование независимой тестовой выборки обеспечивает объективную оценку качества;
- алгоритм обладает низкой вычислительной сложностью.

**Ограничения:**
- чувствительность к освещению;
- ограниченная устойчивость к сложным атакам.

**Рекомендации:**
- использовать в учебных и прототипных системах;
- для практического применения комбинировать с глубинными моделями и мультимодальными данными.
